# Model Deployment

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 16-Bit Precision
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16
).to(device)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Prompt Technique

In [ ]:
fallacy_text = "America is the best place to live, because it's better than any other country."
  cot = f"""
    - You are a fallacy expert.
    - Question:

      - Fallacy Text: {fallacy_text}

        - Possible Fallacies:
          - Fallacy of Credibility (Level 1):
            - Abusive Ad Hominem (Level 2): Attacking the character of the person making an argument rather than the argument itself.
            - Ad Populum (Level 2): Asserting that a proposition is true because many people believe it to be true.
            - Appeal to False Authority (Level 2): Using an authority figure in a field unrelated to the argument as evidence for the argument.
            - Appeal to Nature (Level 2): Arguing that something is good or right because it is natural.
            - Appeal to Tradition (Level 2): Claiming that something is correct or better simply because it is older or traditional.
            - Guilt by Association (Level 2): Discrediting an argument by associating it with an undesirable person or group.
            - Tu Quoque (Level 2): Deflecting criticism by accusing the critic of the same issue.

          - Fallacy of Logic (Level 1):
            - Causal Oversimplification (Level 2): Attributing a complex issue to a single cause.
            - Circular Reasoning (Level 2): Supporting a statement by simply repeating it in different words.
            - Equivocation (Level 2): Using a word with multiple meanings in an argument, misleading the audience.
            - False Analogy (Level 2): Comparing two things that are not truly comparable.
            - False Causality (Level 2): Assuming that because one event follows another, the first event caused the second.
            - False Dilemma (Level 2): Presenting two options as the only possible choices, ignoring other alternatives.
            - Hasty Generalization (Level 2): Making a broad generalization based on limited or insufficient evidence.
            - Slippery Slope (Level 2): Arguing that a minor action will lead to major and often disastrous consequences.
            - Straw Man (Level 2): Misrepresenting someone's argument to make it easier to attack.
            - Fallacy of Division (Level 2): Assuming that what is true of the whole is also true of its parts.

          - Appeal to Emotion (Level 1):
            - Appeal to Positive Emotion (Level 2): Attempting to win an argument by eliciting positive feelings rather than presenting facts.
            - Appeal to Anger (Level 2): Using anger to persuade an audience to accept a conclusion.
            - Appeal to Fear (Level 2): Using fear to influence the audience’s perception or behavior.
            - Appeal to Pity (Level 2): Manipulating an audience's compassion or pity to gain acceptance of an argument.
            - Appeal to Ridicule (Level 2): Mocking an argument to make it appear ridiculous and unworthy of consideration.
            - Appeal to Worse Problem (Level 2): Dismissing an issue by stating that there are worse problems that need attention.

      - Task Description:
        - Detect the fallacy/fallacies in the given 'Fallacy Text' above.
        - If you are unsure or cannot find a fallacy, give the best guess you have.

    - Let's think step by step to identify the fallacy. First give a reasoning path and then output the answer in the output format.

    - Output Format:
      - Only output Level 2 fallacy types.
      - Output in the format: **OUTPUT START** [{{"text": "FALLACY TEXT", "class": "FALLACY TYPE"}}, {{"text": "FALLACY TEXT", "class": "FALLACY TYPE"}}, ...] **OUTPUT END**, where FALLACY TEXT is the excerpt of the text that has the fallacy and FALLACY TYPE is the Level 2 type
      - If there are multiple fallacies within one text, handle them as two seperate objects within the array.
      - Do not forget to put the **OUTPUT START** and **OUTPUT END** delimiters
  """

messages = [
    {"role": "user", "content": cot}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=10000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 
  - Question:

     - Fallacy Text: America is the best place to live, because it's better than any other country.

      - Possible Fallacies:
        - Fallacy of Credibility:
          - Abusive Ad Hominem: Attacking the character of the person making an argument rather than the argument itself.
          - Ad Populum: Asserting that a proposition is true because many people believe it to be true.
          - Appeal to False Authority: Using an authority figure in a field unrelated to the argument as evidence for the argument.
          - Appeal to Nature: Arguing that something is good or right because it is natural.
          - Appeal to Tradition: Claiming that something is correct or better simply because it is older or traditional.
          - Guilt by Association: Discrediting an argument by associating it with an undesirable person or group.
          - Tu Quoque: Deflecting criticism by accusing the critic of the same issue.
        - Fallacy of Logic:
          

# Multi-Label

In [ ]:
from collections import Counter
import re
import numpy as np
import json
import ast

def get_model_response(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    clean = decoded[0].split('[/INST]')[-1].strip()
    print(clean)
    return clean

def extract_fallacies(response):
    pattern = r"(?s)\*\*OUTPUT\s*START\*\*\s*(.*?)\s*\*\*OUTPUT\s*END\*\*"
    matches = re.findall(pattern, response)
    fallacies = []
    for match in matches:
        inner_pattern = r"\"class\": \"(.*?)\""
        fallacy_matches = re.findall(inner_pattern, match)
        fallacies.extend(fallacy_matches)
    return fallacies

def extract_objects(response):
    pattern = r"(?s)\*\*OUTPUT\s*START\*\*\s*(.*?)\s*\*\*OUTPUT\s*END\*\*"
    matches = re.findall(pattern, response)

    fallacy_objects = []

    for match in matches:
        fallacy_objects.append(match)

    return fallacy_objects

def run(fallacy_text, n_runs):
  cot = f"""
    - You are a fallacy expert.
    - Question:

      - Fallacy Text: {fallacy_text}

        - Possible Fallacies:
          - Fallacy of Credibility (Level 1):
            - Abusive Ad Hominem (Level 2): Attacking the character of the person making an argument rather than the argument itself.
            - Ad Populum (Level 2): Asserting that a proposition is true because many people believe it to be true.
            - Appeal to False Authority (Level 2): Using an authority figure in a field unrelated to the argument as evidence for the argument.
            - Appeal to Nature (Level 2): Arguing that something is good or right because it is natural.
            - Appeal to Tradition (Level 2): Claiming that something is correct or better simply because it is older or traditional.
            - Guilt by Association (Level 2): Discrediting an argument by associating it with an undesirable person or group.
            - Tu Quoque (Level 2): Deflecting criticism by accusing the critic of the same issue.

          - Fallacy of Logic (Level 1):
            - Causal Oversimplification (Level 2): Attributing a complex issue to a single cause.
            - Circular Reasoning (Level 2): Supporting a statement by simply repeating it in different words.
            - Equivocation (Level 2): Using a word with multiple meanings in an argument, misleading the audience.
            - False Analogy (Level 2): Comparing two things that are not truly comparable.
            - False Causality (Level 2): Assuming that because one event follows another, the first event caused the second.
            - False Dilemma (Level 2): Presenting two options as the only possible choices, ignoring other alternatives.
            - Hasty Generalization (Level 2): Making a broad generalization based on limited or insufficient evidence.
            - Slippery Slope (Level 2): Arguing that a minor action will lead to major and often disastrous consequences.
            - Straw Man (Level 2): Misrepresenting someone's argument to make it easier to attack.
            - Fallacy of Division (Level 2): Assuming that what is true of the whole is also true of its parts.

          - Appeal to Emotion (Level 1):
            - Appeal to Positive Emotion (Level 2): Attempting to win an argument by eliciting positive feelings rather than presenting facts.
            - Appeal to Anger (Level 2): Using anger to persuade an audience to accept a conclusion.
            - Appeal to Fear (Level 2): Using fear to influence the audience’s perception or behavior.
            - Appeal to Pity (Level 2): Manipulating an audience's compassion or pity to gain acceptance of an argument.
            - Appeal to Ridicule (Level 2): Mocking an argument to make it appear ridiculous and unworthy of consideration.
            - Appeal to Worse Problem (Level 2): Dismissing an issue by stating that there are worse problems that need attention.

      - Task Description:
        - Detect the fallacy/fallacies in the given 'Fallacy Text' above.
        - If you are unsure or cannot find a fallacy, give the best guess you have.

    - Let's think step by step to identify the fallacy. First give a reasoning path and then output the answer in the output format.

    - Output Format:
      - Only output Level 2 fallacy types.
      - Output in the format: **OUTPUT START** [{{"text": "FALLACY TEXT", "class": "FALLACY TYPE"}}, {{"text": "FALLACY TEXT", "class": "FALLACY TYPE"}}, ...] **OUTPUT END**, where FALLACY TEXT is the excerpt of the text that has the fallacy and FALLACY TYPE is the Level 2 type
      - If there are multiple fallacies within one text, handle them as two seperate objects within the array.
      - Do not forget to put the **OUTPUT START** and **OUTPUT END** delimiters
  """

  # Generate multiple responses
  responses = [get_model_response(cot) for _ in range(n_runs)]

  fallacies = [extract_fallacies(response) for response in responses]
  objects = [extract_objects(response) for response in responses]

  fallacies_clean = [list(set(sublist)) for sublist in fallacies]

  flattened_fallacies = [fallacy for sublist in fallacies_clean for fallacy in sublist]
  flattened_objects = [obj for sublist in objects for obj in sublist]

  index_mapping = []
  for i, sublist in enumerate(fallacies):
      for fallacy in sublist:
          index_mapping.append((fallacy, i))

  # Majority Vote
  most_common_fallacy = Counter(flattened_fallacies).most_common(1)[0][0]

  print(f"The most consistent fallacy is: {most_common_fallacy}")


  most_common_fallacy_indices = [index for fallacy, index in index_mapping if fallacy == most_common_fallacy]
  most_common_fallacy_objects = [objects[index] for index in most_common_fallacy_indices]
  most_common_fallacy_object = most_common_fallacy_objects[0] if most_common_fallacy_objects else None

  original_string = most_common_fallacy_object[0]
  try:
      obj = ast.literal_eval(original_string)
  except (ValueError, SyntaxError):
      obj = []

  return obj

In [ ]:
import json
# 1. Load in Golden Standard (Key=Text, Value=Class as Array)
with open('golden_dataset.json') as f:
    data = json.load(f)

# 2. Extract the fallacy text
final_results = []
for j, d in enumerate(data):
  print(f"{j+1}/{len(data)}")
  text = list(d.keys())[0]

  # 3. Run inference and get object array
  result = run(text, n_runs=3)

  # 4. Convert to final format
  ## 4.1 Get Class
  positions = []
  classes = []
  for item in result:
    try:
        pred_text = item["text"]
    except KeyError:
       continue

    try:
        pred_class_name = item["class"]
    except KeyError:
        continue


    start_position = text.find(pred_text)
    if start_position != -1:
      end_position = start_position + len(pred_text)
    else:
      start_position = 0
      end_position = 0
    positions.append((start_position, end_position))
    classes.append(pred_class_name)

  ## 4.2 create object
  classes_formated = []
  for i, c in enumerate(classes):
    formatted = {c: [positions[i][0], positions[i][1]]}
    classes_formated.append(formatted)

  final = {
      text: classes_formated
  }
  print()
  print(final)
  final_results.append(final)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1/200


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START**
[{"text": "I keep seeing if an adventure mode needs to exist then why not just start a char at 68. then a couple months after that we'll see post about how people want to start at 90 just so they can kill bosses.", "class": "Slippery Slope"},
{"text": "a few months later we'll see people asking for a\"creative\" mode so they can have access to all items in the game so they don't have to farm, and i dont think that sets a good example as a community for a game that we all love", "class": "False Dilemma"}]
**OUTPUT END**


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

**OUTPUT START**
[
{"text": "What JP doesn't do is circle jerk with the right.", "class": "Guilt by Association"},
{"text": "if we go into politics, we should go with as little bias as possible and that's not what's happening here.", "class": "False Dilemma"}
]
**OUTPUT END**
The most consistent fallacy is: False Dilemma

{'TITLE: Fuck politics\r\nPOST: He *does* talk about politics. What JP *doesn\'t* do is circle jerk with the right. I think that\'s what you meant to say - "can we stop with the right-wing circle jerk?"\r\nPOST: Yeah but i just think if we go into politics, we should go with as little bias as possible and that\'s not what\'s happening here.': [{'Ad Hominem, Tu Quoque': [0, 0]}, {'False Dilemma': [204, 332]}]}
200/200


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START**
[{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Tu Quoque"}]
**OUTPUT END**


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START**
  [{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Tu Quoque"}]
  **OUTPUT END**
**OUTPUT START**
 [{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Tu Quoque"}]
 **OUTPUT END**
The most consistent fallacy is: Tu Quoque

{"TITLE: What are the most cowardly things a person can do? POST: Refusing to acknowledge that you made a mistake and making excuses instead of being better. We're human. We're flawed - all of us.": [{'Tu Quoque': [64, 155]}]}


In [ ]:
json_data = json.dumps(final_results, indent=4)

with open('data.json', 'w') as file:
    file.write(json_data)

# Multi-Class

In [ ]:
from collections import Counter
import re
import numpy as np
import json
import ast

def get_model_response(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    clean = decoded[0].split('[/INST]')[-1].strip()
    print(clean)
    return clean

def extract_fallacies(response):
    pattern = r"(?s)\*\*OUTPUT\s*START\*\*\s*(.*?)\s*\*\*OUTPUT\s*END\*\*"
    matches = re.findall(pattern, response)
    fallacies = []
    for match in matches:
        inner_pattern = r"\"class\": \"(.*?)\""
        fallacy_matches = re.findall(inner_pattern, match)
        fallacies.extend(fallacy_matches)
    return fallacies

def extract_objects(response):
    pattern = r"(?s)\*\*OUTPUT\s*START\*\*\s*(.*?)\s*\*\*OUTPUT\s*END\*\*"
    matches = re.findall(pattern, response)

    fallacy_objects = []

    for match in matches:
        fallacy_objects.append(match)

    return fallacy_objects

def run(fallacy_text, n_runs):
  cot = f"""
    - You are a fallacy expert.
    - Question:

      - Fallacy Text: {fallacy_text}

        - Possible Fallacies:
          - Fallacy of Credibility (Level 1):
            - Abusive Ad Hominem (Level 2): Attacking the character of the person making an argument rather than the argument itself.
            - Ad Populum (Level 2): Asserting that a proposition is true because many people believe it to be true.
            - Appeal to False Authority (Level 2): Using an authority figure in a field unrelated to the argument as evidence for the argument.
            - Appeal to Nature (Level 2): Arguing that something is good or right because it is natural.
            - Appeal to Tradition (Level 2): Claiming that something is correct or better simply because it is older or traditional.
            - Guilt by Association (Level 2): Discrediting an argument by associating it with an undesirable person or group.
            - Tu Quoque (Level 2): Deflecting criticism by accusing the critic of the same issue.

          - Fallacy of Logic (Level 1):
            - Causal Oversimplification (Level 2): Attributing a complex issue to a single cause.
            - Circular Reasoning (Level 2): Supporting a statement by simply repeating it in different words.
            - Equivocation (Level 2): Using a word with multiple meanings in an argument, misleading the audience.
            - False Analogy (Level 2): Comparing two things that are not truly comparable.
            - False Causality (Level 2): Assuming that because one event follows another, the first event caused the second.
            - False Dilemma (Level 2): Presenting two options as the only possible choices, ignoring other alternatives.
            - Hasty Generalization (Level 2): Making a broad generalization based on limited or insufficient evidence.
            - Slippery Slope (Level 2): Arguing that a minor action will lead to major and often disastrous consequences.
            - Straw Man (Level 2): Misrepresenting someone's argument to make it easier to attack.
            - Fallacy of Division (Level 2): Assuming that what is true of the whole is also true of its parts.

          - Appeal to Emotion (Level 1):
            - Appeal to Positive Emotion (Level 2): Attempting to win an argument by eliciting positive feelings rather than presenting facts.
            - Appeal to Anger (Level 2): Using anger to persuade an audience to accept a conclusion.
            - Appeal to Fear (Level 2): Using fear to influence the audience’s perception or behavior.
            - Appeal to Pity (Level 2): Manipulating an audience's compassion or pity to gain acceptance of an argument.
            - Appeal to Ridicule (Level 2): Mocking an argument to make it appear ridiculous and unworthy of consideration.
            - Appeal to Worse Problem (Level 2): Dismissing an issue by stating that there are worse problems that need attention.

      - Task Description:
        - Detect the fallacy in the given 'Fallacy Text' above.
        - If you are unsure or cannot find a fallacy, give the best guess you have.

    - Let's think step by step to identify the fallacy. First give a reasoning path and then output the answer in the output format.

    - Output Format:
      - Only output Level 2 fallacy types.
      - Output in the format: **OUTPUT START** [{{"text": "FALLACY TEXT", "class": "FALLACY TYPE"}}] **OUTPUT END**, where FALLACY TEXT is the excerpt of the text that has the fallacy and FALLACY TYPE is the Level 2 type
      - Only give one fallacy for this text.
      - Do not forget to put the **OUTPUT START** and **OUTPUT END** delimiters
  """

  # Generate multiple responses
  responses = [get_model_response(cot) for _ in range(n_runs)]

  fallacies = [extract_fallacies(response) for response in responses]
  objects = [extract_objects(response) for response in responses]

  fallacies_clean = [list(set(sublist)) for sublist in fallacies]

  flattened_fallacies = [fallacy for sublist in fallacies_clean for fallacy in sublist]
  flattened_objects = [obj for sublist in objects for obj in sublist]

  index_mapping = []
  for i, sublist in enumerate(fallacies):
      for fallacy in sublist:
          index_mapping.append((fallacy, i))

  # Majority Vote
  most_common_fallacy = Counter(flattened_fallacies).most_common(1)[0][0]

  print(f"The most consistent fallacy is: {most_common_fallacy}")


  most_common_fallacy_indices = [index for fallacy, index in index_mapping if fallacy == most_common_fallacy]
  most_common_fallacy_objects = [objects[index] for index in most_common_fallacy_indices]
  most_common_fallacy_object = most_common_fallacy_objects[0] if most_common_fallacy_objects else None

  original_string = most_common_fallacy_object[0]
  try:
      obj = ast.literal_eval(original_string)
  except (ValueError, SyntaxError):
      obj = []

  return obj

In [ ]:
import json
# 1. Load in Golden Standard (Key=Text, Value=Class as Array)
with open('golden_dataset.json') as f:
    data = json.load(f)

# 2. Extract the fallacy text
final_results_mc = []
for j, d in enumerate(data):
  print(f"{j+1}/{len(data)}")
  text = list(d.keys())[0]

  # 3. Run inference and get object array
  result = run(text, n_runs=3)

  # 4. Convert to final format
  ## 4.1 Get Class
  if (len(result) != 0):
    class_name = result[0].get("class", "")
  else:
    class_name = ""

  final = {
      "text": text,
      "class": class_name
  }

  print()
  print(final)
  final_results_mc.append(final)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1/200


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START** [{"text": "keep seeing if an adventure mode needs to exist then why not just start a char at 68. then a couple months after that we'll see post about how people want to start at 90 just so they can kill bosses.", "class": "Slippery Slope"}] **OUTPUT END**


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

**OUTPUT START** [{"text": "TITLE: Fuck politics\nPOST: He does talk about politics. What JP doesn't do is circle jerk with the right.\nPOST: Yeah but i just think if we go into politics, we should go with as little bias as possible and that's not what's happening here.", "class": "False Dilemma"}] **OUTPUT END**
The most consistent fallacy is: Tu Quoque

{'text': 'TITLE: Fuck politics\r\nPOST: He *does* talk about politics. What JP *doesn\'t* do is circle jerk with the right. I think that\'s what you meant to say - "can we stop with the right-wing circle jerk?"\r\nPOST: Yeah but i just think if we go into politics, we should go with as little bias as possible and that\'s not what\'s happening here.', 'class': 'Tu Quoque'}
200/200


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START** [{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Hasty Generalization"}] **OUTPUT END**

Explanation:
The text above makes a hasty generalization by stating that refusing to acknowledge a mistake and making excuses is the most cowardly thing a person can do. This conclusion relies on a limited and potentially biased perspective, and fails to consider the complexity and nuance of various situations and individual motivations.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**OUTPUT START** [{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Hasty Generalization"}] **OUTPUT END**

Explanation:

The text makes a broad generalization about the significance of not acknowledging mistakes and making excuses instead, without providing any specific evidence or context. This is an example of a Hasty Generalization fallacy.
**OUTPUT START** [{"text": "Refusing to acknowledge that you made a mistake and making excuses instead of being better.", "class": "Hasty Generalization"}] **OUTPUT END**

Explanation: The text makes a broad generalization that refusing to acknowledge mistakes and making excuses instead is the most cowardly thing a person can do, even though there might be many other actions or behaviors that could be considered more cowardly.
The most consistent fallacy is: Hasty Generalization

{'text': "TITLE: What are the most cowardly things a person can do? POST: Refusing to acknowledge that y

In [ ]:
json_data = json.dumps(final_results_mc, indent=4)

with open('data_mc.json', 'w') as file:
    file.write(json_data)